# MUN Chatbot

In [1]:
! pip install -q transformers faiss-cpu gradio accelerate
! pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 126.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56

In [2]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM, BitsAndBytesConfig
import faiss, json, numpy as np
from sentence_transformers import SentenceTransformer
import gradio as gr
import torch

## Text Loading

In [3]:
with open("data/typhus_docs.json", encoding="utf-8") as f:
    docs = json.load(f)

chunks = [d["chunk"] for d in docs]
titles = [d["title"] for d in docs]

## Model Using

In [4]:
model_name = "HuggingFaceH4/zephyr-7b-alpha"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"  # let HF place it on GPU if possible
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
embedder    = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Storing in FAISS

In [5]:
INDEX_PATH = "data/typhus_index.faiss"
try:
    index = faiss.read_index(INDEX_PATH)
except (RuntimeError, OSError):
    emb = embedder.encode(chunks, convert_to_numpy=True).astype("float32")
    index = faiss.IndexFlatL2(emb.shape[1])
    index.add(emb)
    faiss.write_index(index, INDEX_PATH)
    print(f"> Nouvel index écrit dans {INDEX_PATH}")


> Nouvel index écrit dans data/typhus_index.faiss


## Query + LLM Response

In [6]:
def clip(text, max_chars=450):
    return text if len(text) <= max_chars else text[:max_chars] + " ..."

def answer_query(question: str, top_k: int = 5) -> str:
    # 1) recherche FAISS
    q_emb = embedder.encode([question]).astype("float32")
    _, I  = index.search(q_emb, top_k)

    # 2) construire le prompt avec titres + extraits
    sources_txt = []
    picked_titles = []                     # ⬅️  on stocke les titres choisis
    for idx in I[0]:
        title   = titles[idx]
        picked_titles.append(title)        # ⬅️
        passage = clip(chunks[idx])
        sources_txt.append(f"[{title}]\n{passage}")

    prompt = (
        "You are a helpful assistant for Model UN members. Your task is to answer questions "
        "based only on the provided sources.\n\n"
        "Read the sources carefully and answer the user’s question using your own words. "
        "Limit your response to 2–4 complete sentences and conclude your answer clearly. "
        "Do not invent information or include anything not found in the sources.\n\n"
        "Sources:\n" +
        "\n\n".join(sources_txt) +
        f"\n\nQuestion: {question}\nAnswer:"
    )


    # 3) génération GPT‑2
    out = generator(
        prompt,
        max_new_tokens=150,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2,  # Higher = less repetition
        do_sample=True
    )[0]["generated_text"]

    answer_text = out.split("Answer:")[-1].strip()

    # 4) ajouter la liste des titres
    unique_titles = list(dict.fromkeys(picked_titles))      # garde l’ordre, évite doublons
    sources_line  = "Sources: " + "; ".join(unique_titles)

    return f"{answer_text}\n\n{sources_line}"


## Interface

In [7]:
def chatbot(query):
    return answer_query(query)

gr.Interface(fn=chatbot, inputs="text", outputs="text").launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6e41929515ddd68178.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
